In [ ]:
import cv2
import json
import numpy as np
import random
import math
import os

# --- 補助関数 ---
def compute_centroid(points):
    """与えられたキーポイントのリスト [(x, y), ...] の重心を返す"""
    pts = np.array(points)
    return np.mean(pts, axis=0)

def compute_bbox_center(bbox):
    """bbox [x, y, w, h] の中心点 (cx, cy) を返す"""
    x, y, w, h = bbox
    return (x + w / 2, y + h / 2)

def euclidean_distance(pt1, pt2):
    """2点間のユークリッド距離を計算"""
    return math.sqrt((pt1[0] - pt2[0]) ** 2 + (pt1[1] - pt2[1]) ** 2)

def get_random_color():
    """明るすぎないランダムなBGRカラーを生成（各チャンネル 0～200）"""
    return (random.randint(0, 200), random.randint(0, 200), random.randint(0, 200))

def get_purple_color():
    """紫っぽいランダムなBGRカラーを生成
       （赤と青を高め、緑を低めに設定）"""
    red = random.randint(150, 250)
    blue = random.randint(150, 250)
    green = random.randint(0, 100)
    return (blue, green, red)  # OpenCVはBGRの順

def get_green_color():
    """緑っぽいランダムなBGRカラーを生成
       （緑を高め、赤と青を低めに設定）"""
    green = random.randint(100, 250)
    red = random.randint(0, 100)
    blue = random.randint(0, 100)
    return (blue, green, red)  # OpenCVはBGRの順

# --- 1. データの読み込み ---
# JSONファイル（姿勢キーポイントデータ）の読み込み
json_path = '../../ground_truth/Outdoor/IMG_0104_.json'
with open(json_path, 'r') as f:
    json_data = json.load(f)

# 各フレームの姿勢データが記録されている前提（例："frame_0", "frame_1", ...）
annotations = json_data['annotations']

# MOTファイルの読み込み
mot_path = '../../ground_truth/Outdoor/MOT_files/IMG_0104.txt'
mot_data = {}
with open(mot_path, 'r') as f:
    for line in f:
        parts = line.strip().split(',')
        if len(parts) < 6:
            continue
        # フレーム番号は1-indexedと仮定し、0-indexedに変換
        frame_idx = int(parts[0]) - 1
        if frame_idx >= 3600:
            continue  # 最初の3600フレームのみ使用
        track_id = int(parts[1])
        bbox = list(map(float, parts[2:6]))  # [x, y, width, height]
        if frame_idx not in mot_data:
            mot_data[frame_idx] = {}
        mot_data[frame_idx][track_id] = bbox

# --- 2. 対応付けの実施（frame_0 のデータを対象） ---
mot_frame0 = mot_data.get(0, {})
pose_frame0 = annotations.get("frame_0", {})

# 各姿勢アノテーションの重心を計算
pose_centroids = {}  # key: pose_key, value: (cx, cy)
for pose_key, points in pose_frame0.items():
    pose_centroids[pose_key] = compute_centroid(points)

# 各bboxの中心点を計算
bbox_centers = {}  # key: track_id, value: (cx, cy)
for track_id, bbox in mot_frame0.items():
    bbox_centers[track_id] = compute_bbox_center(bbox)

# 重心間の距離を基に、最も近い組み合わせで1対1の対応付けを実施（グリーディ方式）
unmatched_bbox = set(bbox_centers.keys())
track_to_pose = {}    # 対応付け結果：track_id -> pose_key

for pose_key, pose_center in pose_centroids.items():
    best_track = None
    best_dist = float('inf')
    for track_id in unmatched_bbox:
        dist = euclidean_distance(pose_center, bbox_centers[track_id])
        if dist < best_dist:
            best_dist = dist
            best_track = track_id
    if best_track is not None:
        track_to_pose[best_track] = pose_key
        unmatched_bbox.remove(best_track)

# --- 3. 各ペアに対して色を割り当て ---
track_colors = {}  # track_id -> BGR色
for track_id in track_to_pose.keys():
    if track_id in {1, 4, 5}:
        track_colors[track_id] = get_purple_color()
    elif track_id in {3, 6, 8}:
        track_colors[track_id] = get_green_color()
    else:
        track_colors[track_id] = get_random_color()

# --- 4. 動画への描画 ---
video_path = '../../videos/Outdoor/top/IMG_0104_.mp4'
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 出力動画のパス（../videos/Outdoor/top は既に存在）
output_path = '../../videos/Outdoor/top/GT_drawn.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0
while cap.isOpened() and frame_count < 3600:
    ret, frame = cap.read()
    if not ret:
        break

    # --- MOT bbox の描画 ---
    if frame_count in mot_data:
        for track_id, bbox in mot_data[frame_count].items():
            x, y, w, h = bbox
            color = (255, 255, 255)  # 対応が無い場合のデフォルト色
            if track_id in track_to_pose:
                color = track_colors[track_id]
            pt1 = (int(x), int(y))
            pt2 = (int(x + w), int(y + h))
            cv2.rectangle(frame, pt1, pt2, color, 2)
            # bbox上のIDは赤色で表示（数字のみ）
            #cv2.putText(frame, f"{track_id}", (int(x), int(y - 5)),
            #            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # --- 各フレームの姿勢キーポイントの描画 ---
    frame_pose = annotations.get(f"frame_{frame_count}", {})
    for track_id, pose_key in track_to_pose.items():
        if pose_key in frame_pose:
            points = frame_pose[pose_key]
            color = track_colors[track_id]
            for x, y in points:
                cv2.circle(frame, (int(x), int(y)), 8, color, -1)

    out.write(frame)
    frame_count += 1

cap.release()
out.release()